In [1]:
import pandas as pd
import numpy  as np

In [2]:
class SMAStockTrader:
    def __init__(self, data_file):
        # Load and preprocess data
        self.df = pd.read_csv(data_file)
        self.df['<time>'] = pd.to_datetime(self.df['<time>'])
        self.df['Trade Type'] = ""
        self.df['Entry Price'] = 0
        self.df['Exit Price'] = 0
        self.df['Trade Duration'] = pd.Timedelta(0)
        self.df['Profit/Loss'] = ""
        
        # Strategy parameters
        self.stop_loss = 20
        self.target = 30
        self.trade_active = False
        self.entry_price = 0
        self.trade_type = ''
        self.trade_start_time = None
        self.trades = []

    def execute_trades(self):
        for i in range(1, len(self.df)):
            current_row = self.df.iloc[i]
            previous_row = self.df.iloc[i - 1]

            current_sma = current_row['SMA_value']
            current_close = current_row['<close>']
            previous_sma = previous_row['SMA_value']
            previous_close = previous_row['<close>']
            current_time = current_row['<time>']

            if not self.trade_active:
                if previous_sma >= previous_close and current_sma < current_close:
                    self.entry_price = current_close
                    self.trade_type = 'Long'
                    self.trade_start_time = current_time
                    self.trade_active = True

                elif previous_sma < previous_close and current_sma >= current_close:
                    self.entry_price = current_close
                    self.trade_type = 'Short'
                    self.trade_start_time = current_time
                    self.trade_active = True

            if self.trade_active:
                if (current_close - self.entry_price) >= self.target:
                    self.exit_trade(current_close, current_time, 'Profit')

                elif (self.entry_price - current_close) >= self.stop_loss:
                    self.exit_trade(current_close, current_time, 'Loss')

        # Exit any remaining open trades at the end of the day
        last_trade_row = self.df.iloc[-1]
        if self.trade_active:
            self.exit_trade(last_trade_row['<close>'], last_trade_row['<time>'], 'End of Day')

    def exit_trade(self, exit_price, exit_time, trade_result):
        trade_duration = exit_time - self.trade_start_time
        self.trades.append([self.entry_price, exit_price, trade_duration, self.trade_type, trade_result])
        self.trade_active = False

    def calculate_metrics(self):
        total_profit = 0
        total_loss = 0
        total_trades = len(self.trades)
        successful_trades = 0

        for trade in self.trades:
            entry_price, exit_price, _, _, trade_result = trade
            trade_profit = exit_price - entry_price if trade_result == 'Profit' else entry_price - exit_price

            if trade_result == 'Profit':
                successful_trades += 1

            if trade_profit > 0:
                total_profit += trade_profit
            else:
                total_loss += trade_profit

        metrics = {
            'Total Profit': total_profit,
            'Total Loss': total_loss,
            'Success Rate': (successful_trades / total_trades) * 100,
            'Total Trades': total_trades
        }

        return metrics

    def export_trades_to_csv(self, filename):
        trades_df = pd.DataFrame(self.trades, columns=['Entry Price', 'Exit Price', 'Trade Duration', 'Trade Type', 'Profit/Loss'])
        trades_df.to_csv(filename, index=False)

if __name__ == "__main__":
    # Create an instance of the SMAStockTrader class with your data file
    trader = SMAStockTrader("/kaggle/input/nazar-bank-nifty/banknifty_data.csv")
    
    # Execute trades based on the strategy
    trader.execute_trades()
    
    # Calculate and print metrics
    metrics = trader.calculate_metrics()
    print("Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value}")
    
    # Export trades to a CSV file
    trader.export_trades_to_csv("trades.csv")

Metrics:
Total Profit: 2140.30000000001
Total Loss: 0.0
Success Rate: 48.0
Total Trades: 50


## The provided Python code defines a class called SMAStockTrader, which implements a Simple Moving Average (SMA) trading strategy on a given dataset of stock market prices. Here's a short summary of the code:

The code initializes the trader with the dataset and pre-processes it, setting up necessary data columns and strategy parameters.

The execute_trades method iterates through the dataset, identifying entry and exit points for trades based on the SMA strategy. It tracks open trades, entry prices, and trade durations.

The exit_trade method is used to exit a trade, calculating trade duration and recording the trade's result (profit, loss, or end of day).

The calculate_metrics method computes trading metrics such as total profit, total loss, success rate, and total number of trades based on the recorded trades.

The export_trades_to_csv method exports the recorded trades to a CSV file.

In the main block, an instance of SMAStockTrader is created, trades are executed, metrics are calculated and printed, and the trades are exported to a CSV file.



Overall, this code provides a modular and object-oriented approach to backtesting a stock trading strategy, making it easy to analyze the strategy's performance and export the results for further evaluation.

In [3]:
# This is the result of the data(banknifty_data.csv)
pd.read_csv("/kaggle/working/trades.csv")

,Entry Price,Exit Price,Trade Duration,Trade Type,Profit/Loss
0,39139.0,39175.3,0 days 00:03:00,Short,Profit
1,39152.3,39183.4,0 days 00:02:00,Short,Profit
2,39139.9,39194.9,0 days 00:09:00,Short,Profit
3,39172.8,39106.0,0 days 00:05:00,Short,Loss
4,39193.7,39257.7,0 days 00:01:00,Long,Profit
5,39245.8,39199.6,0 days 00:06:00,Short,Loss
6,39249.8,39283.4,0 days 00:01:00,Long,Profit
7,39246.9,39218.8,0 days 00:01:00,Short,Loss
8,39264.9,39233.8,0 days 00:11:00,Long,Loss
9,39264.5,39236.4,0 days 00:02:00,Long,Loss


## BankNifty Option Data

In [4]:
import pandas as pd
import numpy as np

In [5]:
class SMAStockTrader:
    def __init__(self, data_file):
        # Load and preprocess data
        self.df = pd.read_excel(data_file)
        self.df['<date>'] = pd.to_datetime(self.df['<date>'])
        self.df['<time>'] = pd.to_datetime(self.df['<time>'], format='%H:%M:%S').dt.time
        self.df['<datetime>'] = self.df['<date>'] + pd.to_timedelta(self.df['<time>'].astype(str))
        self.df['Trade Type'] = ""
        self.df['Entry Price'] = 0
        self.df['Exit Price'] = 0
        self.df['Trade Duration'] = pd.Timedelta(0)
        self.df['Profit/Loss'] = ""
        
        # Strategy parameters
        self.stop_loss = 20
        self.target = 30
        self.trade_active = False
        self.entry_price = 0
        self.trade_type = ''
        self.trade_start_time = None
        self.trades = []

    def execute_trades(self):
        for i in range(1, len(self.df)):
            current_row = self.df.iloc[i]
            previous_row = self.df.iloc[i - 1]

            current_close = current_row['<close>']
            previous_close = previous_row['<close>']
            current_time = current_row['<datetime>']

            if not self.trade_active:
                if previous_close > current_close:
                    self.entry_price = current_close
                    self.trade_type = 'Long'
                    self.trade_start_time = current_time
                    self.trade_active = True

                elif previous_close < current_close:
                    self.entry_price = current_close
                    self.trade_type = 'Short'
                    self.trade_start_time = current_time
                    self.trade_active = True

            if self.trade_active:
                if (current_close - self.entry_price) >= self.target:
                    self.exit_trade(current_close, current_time, 'Profit')

                elif (self.entry_price - current_close) >= self.stop_loss:
                    self.exit_trade(current_close, current_time, 'Loss')

        # Exit any remaining open trades at the end of the day
        last_trade_row = self.df.iloc[-1]
        if self.trade_active:
            self.exit_trade(last_trade_row['<close>'], last_trade_row['<datetime>'], 'End of Day')

    def exit_trade(self, exit_price, exit_time, trade_result):
        trade_duration = exit_time - self.trade_start_time
        self.trades.append([self.entry_price, exit_price, trade_duration, self.trade_type, trade_result])
        self.trade_active = False

    def calculate_metrics(self):
        total_profit = 0
        total_loss = 0
        total_trades = len(self.trades)
        successful_trades = 0

        if total_trades > 0:  # Check if there are any trades
            for trade in self.trades:
                entry_price, exit_price, _, _, trade_result = trade
                trade_profit = exit_price - entry_price if trade_result == 'Profit' else entry_price - exit_price

                if trade_result == 'Profit':
                    successful_trades += 1

                if trade_profit > 0:
                    total_profit += trade_profit
                else:
                    total_loss += trade_profit

            metrics = {
                'Total Profit': total_profit,
                'Total Loss': total_loss,
                'Success Rate': (successful_trades / total_trades) * 100,
                'Total Trades': total_trades
            }
        else:
            metrics = {
                'Total Profit': total_profit,
                'Total Loss': total_loss,
                'Success Rate': 0,  # Set success rate to 0 if there are no trades
                'Total Trades': total_trades
            }

        return metrics

    def export_trades_to_csv(self, filename):
        trades_df = pd.DataFrame(self.trades, columns=['Entry Price', 'Exit Price', 'Trade Duration', 'Trade Type', 'Profit/Loss'])
        trades_df.to_csv(filename, index=False)

if __name__ == "__main__":
    trader = SMAStockTrader("/kaggle/input/banknifty-option-nazar/option_contract_data.xlsx")
    trader.execute_trades()
    metrics = trader.calculate_metrics()
    trader.export_trades_to_csv("trades_bnk_opt.csv")

    # Print and/or save the metrics as needed
    print("Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

Metrics:
Total Profit: 1585.9999999999993
Total Loss: -1.0499999999999972
Success Rate: 28.000000000000004
Total Trades: 50


## 1. Data Loading and Preprocessing

The code starts by loading financial data from an Excel file into a Pandas DataFrame.
It converts date and time columns into appropriate datetime formats and creates a combined datetime column.
## 2. SMA Trading Strategy

The code defines an SMA-based trading strategy with parameters for stop loss and target profit.
It iterates through the financial data to identify potential trade entry and exit points based on SMA conditions.
Trades are executed when SMA conditions are met, and exit conditions are checked to either book a profit or limit losses.
## 3. Trade Management

The code manages trade-related variables such as trade type, entry price, exit price, trade duration, and profit/loss.
Trades are recorded in a list for further analysis.
## 4. Metrics Calculation

After all trades are executed, the code calculates trading metrics such as total profit, total loss, success rate, and total number of trades.
## 5. Export to CSV

The code exports trade details, including entry price, exit price, trade duration, trade type, and profit/loss, to a CSV file.
## 6. Usage

The code can be used by creating an instance of the SMAStockTrader class and calling its methods to execute trades, calculate metrics, and export trade data.
Overall, this code provides a flexible and modular framework for backtesting an SMA trading strategy on financial data and analyzing the trading performance.